In [2]:
# get imports # import files
from threading import Thread
import queue as que
import pandas as pd
import os, pickle
import nltk
import math
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
from collections import Counter 

stemmer = SnowballStemmer("english")
cachedStopWords = stopwords.words("english")

In [3]:
#methods
def save_it_all(obj, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    
def load_objects(file):
    with open(file, 'rb') as input:
        return pickle.load(input)
    
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def seperator(docs):
    print(len(docs))
    txt = []
    typesInfo = []
    docsInfo = []
    for line in docs:
        words = line.split()
        content = (" ").join(words[1:])
        item = (", ").join([words[0], content])
        txt.append(item)
        typesInfo.append(words[0])
        docsInfo.append(content)
    return txt, typesInfo, docsInfo

def write(data):
    print(type(data))
    with open('./training_data.csv', 'w') as f:
        for item in data:
            f.write("%s\n" % item)
            
# methods
def read(fileName):
    with open(fileName) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    return content

def save_it_all(obj, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    
def load_objects(file):
    with open(file, 'rb') as input:
        return pickle.load(input)
    
def vocabDict(vocab, doc, n): # handles empty word sets
    nk = 0
    # n = the number of word postions for this document type
    probability = math.log(nk + 1) - math.log(n + len(vocab))

    newDict = {"count": 0, "probability":probability}
    vdict = dict((el,newDict) for el in vocab)
    return vdict

def wordCount(doc, vocab, n): # get actual word count for each word and create dict frame to track the info
    counts = dict(Counter(doc.tolist()[0].split()))
    for key, value in counts.items():
        nk = value
        probability = math.log(nk + 1) - math.log(n + len(vocab))
        newDict = {"count": nk, "probability": probability}
        counts[key] = newDict
    emptyVocab = vocabDict(vocab, doc, n)
    dicts = [merge_two_dicts(emptyVocab, counts)]
    return dicts
        
def wordProbs(dataFrame):
    for docType in dataFrame["Type"]:
        wordCountDict = dataFrame["WordCount"][dataFrame["Type"]==docType].tolist()[0]
        
        for key in wordCountDict:
            if dataFrame["WordCount"][dataFrame["Type"]==docType].tolist()[0][key] is int:
                nk = dataFrame["WordCount"][dataFrame["Type"]==docType].tolist()[0][key]
                n = dataFrame["WordPositions"][dataFrame["Type"]==docType].tolist()[0]
                probability = math.log(nk + 1) - math.log(n + len(vocabStopStemmingEdit))
                newDict = {"count": nk, "probability":probability}
                dataFrame["WordCount"][dataFrame["Type"]==docType].tolist()[0][key] = newDict
    return dataFrame

def saveFrame(df, name):
    df.to_csv(name+".csv", index=False, sep=",", header=True)
    save_it_all(df, name+".pkl")

In [4]:
# # load in files 
# # read file line by line 
# fileName = "../forumTraining.data"
# data = read(fileName)
# len(data)
# sep_data, typesInfo, docsInfo = seperator(data)
# # write(sep_data) not needed the raw csv it makes IS BROKEN
# # print(len(list(set(typesInfo))))
# # print(list(set(typesInfo)))
# dataFrameRaw = pd.DataFrame({"Type": typesInfo, "Document": docsInfo})
# saveFrame(dataFrameRaw, "raw_training_data.csv")

In [5]:
raw_training_data = pd.read_csv("./raw_training_data.csv", sep=",")
raw_training_data.head()

,Type,Document
0,atheism,alt atheism faq atheist resources archive name...
1,atheism,alt atheism faq introduction to atheism archiv...
2,atheism,re gospel dating in article mimsy umd edu mang...
3,atheism,re university violating separation of church s...
4,atheism,re soc motss et al princeton axes matching fun...


### Trianing data raw is held in the dataframe "raw_training_data"

In [6]:
# create the raw vocabulary set:
raw_vocabulary = " ".join(raw_training_data["Document"].tolist()) # concatinates all documents in the data frame
raw_vocabulary = list(set(raw_vocabulary.split()))
print(len(raw_vocabulary))
save_it_all(raw_vocabulary, "./raw_vocabulary.pkl")

73712


### The raw vocabulary is saved in the above cell and refrenced as "raw_vocabulary" 

# This is the learning step: make it dynamic!
For each class cj (document type) in C
    
    1. Docsj ← training documents for which the classification is cj
    
    2. Probability estimate of a particular class: P(cj) = |Docsj| / |training documents|
    
    3. Textj ← create a single document per class (concatenate all Docsj)
    
    4. n = total number of word positions in Textj
    
    5. For each word wk in Vocabulary nk = number of times wk occurs in Textj
    
    6. Estimate of word occurrence for particular document type: P(wk | cj) = (nk + 1) / (n + |Vocabulary|)
    
Probability of kth word in vocabulary, given a document of type j

In [7]:
# load in docs you want to use:
## trainign data set for use:
trainFrame = pd.read_csv("./raw_training_data.csv", sep=",")

## vocabulary training set for use:
vocabulary = load_objects("./raw_vocabulary.pkl")

# validate:
print(type(vocabulary))
trainFrame.head()

<class 'list'>


,Type,Document
0,atheism,alt atheism faq atheist resources archive name...
1,atheism,alt atheism faq introduction to atheism archiv...
2,atheism,re gospel dating in article mimsy umd edu mang...
3,atheism,re university violating separation of church s...
4,atheism,re soc motss et al princeton axes matching fun...


In [8]:
# single docs per class
types = list(set(trainFrame["Type"]))
singleFrame = pd.DataFrame({"Type":types , "Document": None})
# singleFrame.head(20)
for docType in singleFrame["Type"]:
    #print(trainFrame["Document"][trainFrame["Type"]==docType])
    singleFrame["Document"][singleFrame["Type"]== docType] = " ".join(trainFrame["Document"][trainFrame["Type"]==docType].tolist())
singleFrame.head(20)

,Type,Document
0,graphics,call for presentations navy sciviz vr seminar ...
1,space,re gravity waves was predicting gravity wave q...
2,guns,re gun control was re we re mad as hell at the...
3,baseball,spring records the orioles pitching staff agai...
4,cryptology,ripem frequently asked questions archive name ...
5,politics,re media horrified at perot investigating bush...
6,religion,re religion and homosexuality magarret magnus ...
7,autos,re saturn s pricing policy in article c vir l ...
8,christianity,re sex education in article mar athos rutgers ...
9,electronics,re help with tracking device in article fba e ...


In [9]:
# make a copy of the single frame 
rawDataInfoFrame = singleFrame.copy()

In [10]:
# probability estimate of the class
rawDataInfoFrame["ClassProbability"] = None
for docType in rawDataInfoFrame["Type"]:
    rawDataInfoFrame["ClassProbability"][rawDataInfoFrame["Type"]== docType] = len(trainFrame["Document"][trainFrame["Type"]== docType].tolist())/trainFrame["Type"].count()


In [11]:
rawDataInfoFrame.head()

,Type,Document,ClassProbability
0,graphics,call for presentations navy sciviz vr seminar ...,0.0517135
1,space,re gravity waves was predicting gravity wave q...,0.0525104
2,guns,re gun control was re we re mad as hell at the...,0.04826
3,baseball,spring records the orioles pitching staff agai...,0.0528646
4,cryptology,ripem frequently asked questions archive name ...,0.0526875


In [12]:
# get the word postions:
rawDataInfoFrame["WordPositions"] = None
for docType in rawDataInfoFrame["Type"]:
    rawDataInfoFrame["WordPositions"][rawDataInfoFrame["Type"]== docType] = len(rawDataInfoFrame["Document"][rawDataInfoFrame["Type"]== docType].tolist()[0].split())
    


In [13]:
rawDataInfoFrame.head()

,Type,Document,ClassProbability,WordPositions
0,graphics,call for presentations navy sciviz vr seminar ...,0.0517135,123428
1,space,re gravity waves was predicting gravity wave q...,0.0525104,168481
2,guns,re gun control was re we re mad as hell at the...,0.04826,191723
3,baseball,spring records the orioles pitching staff agai...,0.0528646,119407
4,cryptology,ripem frequently asked questions archive name ...,0.0526875,219507


In [14]:
# word count 
rawDataInfoFrame["WordCount"] = None
for docType in rawDataInfoFrame["Type"]:
    n = rawDataInfoFrame["WordPositions"][rawDataInfoFrame["Type"] == docType]
    rawDataInfoFrame["WordCount"][rawDataInfoFrame["Type"] == docType] = wordCount(rawDataInfoFrame["Document"][rawDataInfoFrame["Type"]==docType], vocabulary, n)
    

In [15]:
rawDataInfoFrame.head()
# rawDataInfoFrame["WordCount"][rawDataInfoFrame["Type"] == "autos"].tolist()

,Type,Document,ClassProbability,WordPositions,WordCount
0,graphics,call for presentations navy sciviz vr seminar ...,0.0517135,123428,"{'surface': {'count': 70, 'probability': -7.92..."
1,space,re gravity waves was predicting gravity wave q...,0.0525104,168481,"{'surface': {'count': 78, 'probability': -8.02..."
2,guns,re gun control was re we re mad as hell at the...,0.04826,191723,"{'surface': {'count': 2, 'probability': -11.39..."
3,baseball,spring records the orioles pitching staff agai...,0.0528646,119407,"{'surface': {'count': 1, 'probability': -11.47..."
4,cryptology,ripem frequently asked questions archive name ...,0.0526875,219507,"{'surface': {'count': 1, 'probability': -11.89..."


In [16]:
saveFrame(rawDataInfoFrame, "./raw_data_info_frame_V1")

In [19]:
rawDataInfoFrameEdit = rawDataInfoFrame.drop(columns=["WordPositions"])

In [21]:
saveFrame(rawDataInfoFrameEdit, "./raw_data_info_frame_V2")
rawDataInfoFrameEdit.head()

,Type,Document,ClassProbability,WordCount
0,graphics,call for presentations navy sciviz vr seminar ...,0.0517135,"{'surface': {'count': 70, 'probability': -7.92..."
1,space,re gravity waves was predicting gravity wave q...,0.0525104,"{'surface': {'count': 78, 'probability': -8.02..."
2,guns,re gun control was re we re mad as hell at the...,0.04826,"{'surface': {'count': 2, 'probability': -11.39..."
3,baseball,spring records the orioles pitching staff agai...,0.0528646,"{'surface': {'count': 1, 'probability': -11.47..."
4,cryptology,ripem frequently asked questions archive name ...,0.0526875,"{'surface': {'count': 1, 'probability': -11.89..."
